# Analyzing borrowers’ risk of defaulting

We have a data with personal information about people who have a loan in the bank. Based on this information we have to prepare a report for a bank’s loan division. The task is to find out if a customer’s marital status and number of children have an impact on whether they will default on a loan. The bank already has some data on customers’ credit worthiness. Results of the research will be use for building a credit score for a potential customer.

## Open the data file and have a look at the general information. 

In [1]:
# read in the data
import pandas as pd
data = pd.read_csv('/datasets/credit_scoring_eng.csv')
print(data)

       children  days_employed  dob_years            education  education_id  \
0             1   -8437.673028         42    bachelor's degree             0   
1             1   -4024.803754         36  secondary education             1   
2             0   -5623.422610         33  Secondary Education             1   
3             3   -4124.747207         32  secondary education             1   
4             0  340266.072047         53  secondary education             1   
...         ...            ...        ...                  ...           ...   
21520         1   -4529.316663         43  secondary education             1   
21521         0  343937.404131         67  secondary education             1   
21522         1   -2113.346888         38  secondary education             1   
21523         3   -3112.481705         38  secondary education             1   
21524         2   -1984.507589         40  secondary education             1   

           family_status  family_status

In [2]:
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
5,0,-926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house
6,0,-2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions
7,0,-152.779569,50,SECONDARY EDUCATION,1,married,0,M,employee,0,21731.829,education
8,2,-6929.865299,35,BACHELOR'S DEGREE,0,civil partnership,1,F,employee,0,15337.093,having a wedding
9,0,-2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family


In [3]:
#check datatype
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [4]:
for i in data.columns.drop(['days_employed', 'total_income']): 
    print(f'Unique values in column{str (i)}: {data[i].unique()}')

Unique values in columnchildren: [ 1  0  3  2 -1  4 20  5]
Unique values in columndob_years: [42 36 33 32 53 27 43 50 35 41 40 65 54 56 26 48 24 21 57 67 28 63 62 47
 34 68 25 31 30 20 49 37 45 61 64 44 52 46 23 38 39 51  0 59 29 60 55 58
 71 22 73 66 69 19 72 70 74 75]
Unique values in columneducation: ["bachelor's degree" 'secondary education' 'Secondary Education'
 'SECONDARY EDUCATION' "BACHELOR'S DEGREE" 'some college'
 'primary education' "Bachelor's Degree" 'SOME COLLEGE' 'Some College'
 'PRIMARY EDUCATION' 'Primary Education' 'Graduate Degree'
 'GRADUATE DEGREE' 'graduate degree']
Unique values in columneducation_id: [0 1 2 3 4]
Unique values in columnfamily_status: ['married' 'civil partnership' 'widow / widower' 'divorced' 'unmarried']
Unique values in columnfamily_status_id: [0 1 2 3 4]
Unique values in columngender: ['F' 'M' 'XNA']
Unique values in columnincome_type: ['employee' 'retiree' 'business' 'civil servant' 'unemployed'
 'entrepreneur' 'student' 'paternity / materni

We checked for uniqueness in our dataset. We found out that:

children has a negative value of -1 so we have to deal with it
days_employed have negative value of days. We have to take the absolute value for our analysis to be accurate
education has duplicate data
gender has an 'XNA' entry. Since there is no logical way to decide whether that entry is for a male or female, we would have to drop that row.

In [5]:
data['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [6]:
data['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

### Conclusion

From the Data Preprocessing section, we computed the number of missing values per column and the percentage of missing values per column. We can see that 10% of rows are missing days_employed and total_income. Since the type of data with missing values are quantitative, and the data are "Missing not at random", we proceed to drop those rows with missing values. We drop missing rows because 10% of missing values would not really skew our analysis.

## Data preprocessing

### Processing missing values

In [7]:
data['days_employed'] = data['days_employed'].fillna(0)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [8]:
print("Maximum monthly income: {: .0f}".format(data['total_income'].max ()))
print("Minimum monthly income: {: .0f}".format(data['total_income'].min ()))
print("Ratio of maximum monthly income to minimum income: {: .0f}".format(
     data['total_income'].max() / data['total_income'].min ()))

Maximum monthly income:  362497
Minimum monthly income:  3307
Ratio of maximum monthly income to minimum income:  110


In [9]:
data.groupby('income_type')['total_income'].median()     

income_type
business                       27577.2720
civil servant                  24071.6695
employee                       22815.1035
entrepreneur                   79866.1030
paternity / maternity leave     8612.6610
retiree                        18962.3180
student                        15712.2600
unemployed                     21014.3605
Name: total_income, dtype: float64

In [10]:
data['total_income'] = data[['total_income']].fillna(data.groupby("income_type").transform("median"))

In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [12]:
data['children'] = data['children'].replace(-1, 1)

In [13]:
data['children'].value_counts()

0     14149
1      4865
2      2055
3       330
20       76
4        41
5         9
Name: children, dtype: int64

### Conclusion

All missing values of type "NaN" in columns 'days_employed' and 'total_income' are filled with zeros and median for income_type groups, respectively. Value (-1) in column 'children' is replaced with (1). Since it is not possible to identify the gender of the client in the lines with the data 'xna' according to the available data and we are not faced with research tasks concerning the gender of the client, we will leave the data unchanged.

### Data type replacement

In [14]:
data['days_employed'] = abs(data['days_employed']) 
#data.loc[data['days_employed'] < 0,'days_employed'] = data['days_employed'] * (-1) 

In [15]:
data['days_employed'] = data['days_employed'].astype('int') 
data['total_income'] = data['total_income'].astype('int')

In [16]:
data.dtypes

children             int64
days_employed        int64
dob_years            int64
education           object
education_id         int64
family_status       object
family_status_id     int64
gender              object
income_type         object
debt                 int64
total_income         int64
purpose             object
dtype: object

In [17]:
data[['days_employed', 'total_income']].head()

,days_employed,total_income
0,8437,40620
1,4024,17932
2,5623,23341
3,4124,42820
4,340266,25378


In [18]:
if data[data['days_employed'] < 0]['days_employed'].count() > 0: 
    print('There are negative values in the "days_employed" column')
else:
    print('There are no negative values in the days_employed column.')

There are no negative values in the days_employed column.


# Conclusion

For convenience and clarity of data analysis, the values in the 'days_employed' and 'total_income' columns are replaced with integer values by the .astype ('int') method.

### Processing duplicates

In [19]:
data['family_status'] = data['family_status'].str.lower() 
data['education'] = data['education'].str.lower() 
print('Unique values in the column "family status":', data['family_status'].unique())
print('Unique values in the column "customer education":', data['education'].unique())

Unique values in the column "family status": ['married' 'civil partnership' 'widow / widower' 'divorced' 'unmarried']
Unique values in the column "customer education": ["bachelor's degree" 'secondary education' 'some college'
 'primary education' 'graduate degree']


In [20]:
data.duplicated().sum()

71

In [21]:
data = data.drop_duplicates().reset_index(drop=True)

### Conclusion

Decreasing values are reduced to the general form of writing in lower case. Identified and removed 71 duplicate lines. Possible reasons for the occurrence of duplicates in the human factor, errors when converting the dataframe.

### Categorizing Data

In [22]:
def children_category(row):
    if row == 0:
        return 'no children'
    return 'children'

In [23]:
data['children_status'] = data['children'].apply(children_category)

In [24]:
data['children_status'].value_counts()

no children    14091
children        7363
Name: children_status, dtype: int64

In [25]:
print('Dictionary for determining position')
family_dictionary = data[['family_status_id', 'family_status']].drop_duplicates().reset_index(drop=True)
family_dictionary

Dictionary for determining position


,family_status_id,family_status
0,0,married
1,1,civil partnership
2,2,widow / widower
3,3,divorced
4,4,unmarried


In [26]:
data['family_status_id'].value_counts(sort=False)

0    12339
1     4151
2      959
3     1195
4     2810
Name: family_status_id, dtype: int64

In [27]:
low_income = (data['total_income']).quantile(.33) 
mean_income = (data['total_income']).quantile(.66)

In [28]:
def income(total_income):
    if total_income <= low_income:
        return "low_income"
    if total_income <= mean_income:
        return "average_income"
    return "high_income"

In [29]:
print(f"If the average monthly income is not more than {low_income:.0f}, then the client is defined in the '{income(low_income)}'")
print(f"If the average monthly income is not more than {mean_income:.0f}, then the client is determined to the '{income(mean_income)}'")
print(f"If the average monthly income is more than {mean_income:.0f}, then the client is identified in the '{income(mean_income + 0.1)}'")

If the average monthly income is not more than 18962, then the client is defined in the 'low_income'
If the average monthly income is not more than 27577, then the client is determined to the 'average_income'
If the average monthly income is more than 27577, then the client is identified in the 'high_income'


In [30]:
data['income_group'] = data['total_income'].apply(income)

In [31]:
data['income_group'].value_counts()

average_income    7340
low_income        7097
high_income       7017
Name: income_group, dtype: int64

In [32]:
lemm_list = ['renovation', 'construction', 'wedding', 'housing', 'education', 'car', 'real estate'] 

def purpose_categorize(row):
    for word in lemm_list:
        if word in row:
            return word

In [33]:
data['purpose_category'] = data['purpose'].apply(purpose_categorize)

In [34]:
data['purpose_category'].value_counts()

real estate     4464
car             4306
education       3109
wedding         2324
housing         1298
construction     635
renovation       607
Name: purpose_category, dtype: int64

### Conclusion

The data required to solve the assigned tasks is categorized. Columns have been created to categorize by the presence of children, average monthly income, loan purposes. A dictionary has been compiled to determine the family status of a client.

## Answer these questions

- Is there a relation between having kids and repaying a loan on time?

In [35]:
data_pivot_children = data.pivot_table(index=['children_status'], values='debt', aggfunc='mean')
data_pivot_children.style.format({'debt': '{:.2%}'})

,debt
children_status,
children,9.21%
no children,7.54%


### Conclusion

9.21% of clients in the category "have children" have a loan repayment debt. 7.54% of clients in the "no children" category have loan repayments in arrears. The presence of children increases the likelihood of loan defaults on time.

- Is there a relation between marital status and repaying a loan on time?

In [36]:
data_pivot_family = data.pivot_table(index=['family_status'], values='debt', aggfunc='mean')
data_pivot_family.sort_values(by = 'debt', ascending=False).style.format({'debt': '{:.2%}'})

,debt
family_status,
unmarried,9.75%
civil partnership,9.35%
married,7.55%
divorced,7.11%
widow / widower,6.57%


### Conclusion

The smallest share of non-repayment of loans on time is observed in the category "widower / widow" - 6.57%.
The largest share of non-repayment of loans on time is observed in the categories "not married / not married" and "civil marriage" - 9.75% and 9.35%, respectively.
The seed position significantly affects the loan repayment on time.

- Is there a relation between income level and repaying a loan on time?

In [37]:
data_pivot_income = data.pivot_table(index=['income_group'], values='debt', aggfunc='mean')
data_pivot_income.sort_values(by='debt', ascending=False).style.format({'debt': '{:.2%}'})

,debt
income_group,
average_income,8.64%
low_income,8.16%
high_income,7.52%


### Conclusion

The smallest share of non-repayment of loans on time is observed in the category "high level of income" - 7.52%.
The largest share of non-repayment of loans on time is observed in the category "average income" - 8.64%.
The level of income least of all affects the loan repayment on time is in the category "low income" - 8.16%.

- How do different loan purposes affect on-time repayment of the loan?

In [38]:
data_pivot_purpose = data.pivot_table(index=['purpose_category'], values='debt', aggfunc='mean')
data_pivot_purpose.sort_values(by = 'debt', ascending=False).style.format({'debt': '{:.2%}'})

,debt
purpose_category,
car,9.36%
education,9.26%
wedding,8.00%
real estate,7.53%
housing,7.24%
construction,6.61%
renovation,5.77%


### Conclusion

The smallest share of loan defaults on time is observed in the "house" category - 5.77%. The largest share of non-repayment of loans on time is observed in the category "car" - 9.36% and education - 9.36%.
The purpose of the loan significantly affects its return on time.

## General conclusion

The study analyzed data on the solvency of 21,525 clients, which include such characteristics as the number of children in the family, total work experience, age of the client, level of education, marital status, gender of the client, type of employment, presence / absence of debt to repay loans in past, monthly income, the purpose of obtaining a loan.

The purposes of obtaining the loan were divided into four main categories: real estate, education, car and wedding - based on the goals indicated by the clients. We received 4,284 people for a car loan; 10 762 people for the purpose of a loan for real estate; 3,995 clients for educational purposes; and 2,310 people for the wedding. At the same time, there are no obvious dependencies between the goals of the loan and other factors.

Data categorization was also carried out. Since the most obvious sign by which we can judge the likelihood of a client repaying a debt is his income, it was chosen as a criterion: the higher the income, the more likely the clients are to issue a loan (since his solvency is higher). However, it should be noted that in the absence of other data, we split the income into 5 equal categorical segments; which can influence the adoption of wrong decisions. The boundaries should be discussed with the project customer (bank) and, if necessary, adjusted.

For income categories (probability of obtaining a loan), the following patterns were identified for other factors. For clients without children, the likelihood of a loan is generally medium and low. For clients with children (1 or more), the probability of extradition is distributed evenly. For people with higher education, clients with a high and very high probability of obtaining a loan prevail, and for clients with secondary or primary education - with an average and very low probability. In terms of marital status, there are no significant differences between the categories, except for the “widower / widow” category: for them, the probability of obtaining a loan is mostly medium or low. There are differences by gender: for women, the likelihood of getting a loan is mostly medium or low, and for men it is very high or high. This may not be due to gender differences in the bank's approach, but rather to gender differences in income that have developed in the labor market. For employees and retirees, the likelihood of obtaining a loan is, to a greater extent, medium or low. And for partners and civil servants - high and very high. But for civil servants, the probabilities are more evenly distributed. For the rest of the categories, there is insufficient data to identify patterns.

Then, an analysis of the factors that can influence the loan repayment was carried out. Clients with almost any number of children (or their absence) return the loan in 9 cases out of 100. However, large families (with 5 children) do this in 100 cases out of 100 (although the sample may not be enough for such a conclusion). Nevertheless, the factor of the possibility of debt repayment may not be the number of children, but the availability of appropriate income, as well as the willingness to take responsibility. The same is true for the marital status: those living in a civil marriage or unmarried return the loan less often than married, widowed and divorced. It can also be indirectly related to the willingness to take responsibility or the age of the client. There is no direct relationship between income level and loan repayment, both people with high and low incomes have loan delinquencies (the percentage in both categories is very close). There is a direct relationship between the goals of the loan and its repayment on time: goals associated with higher costs and a later stage in life (real estate, wedding) are characterized by higher rates of debt repayment than the goals of a loan for education and a car.